In [61]:
import sys
sys.path.append('../..')

import pickle
import ml.structures
from ml.dataset.corpus import *
import numpy as np

import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from keras.models import load_model

from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import RMSprop


from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.engine.topology import Input

from sklearn.metrics import accuracy_score
from keras.layers import concatenate
from keras.layers.merge import Concatenate

import keras
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output
DATASET_HOME='../../datasets/'

In [2]:
def make_16(chord):
    if (chord.duration > 0):
        c = 16 / (128 / chord.duration)
    else:
        return 0, [0]
    pre_array = np.array([note.number % 12 for note in chord.notes])
    array = set()
    for el in pre_array:
        array.add(el)
        if len(array) == 4:
            break
    array = np.array(list(array))
    array = np.sort(array)
    if len(array) == 0:
        array = [0]
    if c >= 1:
        return 1, np.tile(array, (int(c), 1))
    else:
        return 0, [0]

In [3]:
def cut_song(song, notes, chords):
    chords_track = []
    notes_track = np.array([])
    flag = 0
    for note in song.tracks[0].chords:
        if (make_16(note)[0] == 0):
            flag = 1
        notes_track = np.append(notes_track, make_16(note)[1])
    for chord in song.tracks[1].chords:
        if (make_16(chord)[0] == 0):
            flag = 1
        chords_track.append(make_16(chord)[1])
    if flag == 1:
        return 0, notes, chords
    return 1, notes_track, chords_track

In [4]:
def norm_array(X, size):
    for j in range(len(X)):
        X[j] = np.unique(np.array(X[j]))[:size]
        array = np.zeros(size-len(X[j])) - 1
        X[j] = np.append(X[j], array)
    X = np.array(X)
    return X

In [5]:
def create_X_Y(chords):
    real_chords = []
    y = []
    for i, subchord in enumerate(chords):
        if i != len(chords) - 1:
            for ssubchord in subchord:
                real_chords.append(ssubchord)
            size = len(subchord)
            for j in range(size):
                y.append(chords[i+1][-1])
    return real_chords, y

In [6]:
def notes_to_np(notes):
    for i in range(len(notes)):
        notes[i] = np.array(notes[i])
    notes = np.array(notes)

In [7]:
def create_unique_chords(X, unique_chords):
    for x in X:
        unique_chords = np.append(unique_chords, np.array(x))
    return unique_chords

In [8]:
def create_coded(X):
    return np.zeros(len(X))


In [9]:
def encode(X, coded_X):
    for j, k in enumerate(X):
        for i, unique_chord in enumerate(unique_chords):
            if np.array_equal(k, unique_chord):
                coded_X[j] = i

In [55]:
model_chords_input = Input((334, ), batch_shape=(1,1))
model_chords_embedding = Embedding(794, 20, input_length=1, batch_input_shape=(1,1))(model_chords_input)
model_notes_input = Input((334, ), batch_shape=(1,1))
model_notes_embedding = Embedding(128, 20, input_length=1, batch_input_shape=(1,1))(model_notes_input)

In [56]:
merged = keras.layers.concatenate([model_chords_embedding, model_notes_embedding])

lstm = LSTM(20, stateful=True)(merged)
model_dense = Dense(794, activation='softmax')(lstm)

merged_model = keras.Model(inputs=[model_chords_input, model_notes_input], outputs=model_dense)
merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [57]:
# model_chords = Sequential()
# model_notes = Sequential()

# model_chords.add(Embedding(794, 20, input_length=1, batch_input_shape=(1,1)))
# model_notes.add(Embedding(128, 20, input_length=1, batch_input_shape=(1,1)))

In [58]:
# merged_model = Sequential()
# merged_model.add(keras.layers.InputLayer(input_tensor=keras.layers.concatenate([model_chords.output, model_notes.output])))
# merged_model.add(LSTM(20, stateful=True))
# merged_model.add(Dense(794, activation='softmax'))
# merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [63]:
corpus = SongCorpus()
corpus.load_from_file(DATASET_HOME+'simple/simple_dataset_SimplifyChordsMapper.pickle')

In [65]:
allX = []
allY = []
for song in tqdm.tqdm_notebook(corpus.songs):
    chords = []
    notes = []
    flag_notes_chords = cut_song(song, notes, chords)
    if (flag_notes_chords[0] == 0) :
        continue
    notes = flag_notes_chords[1]
    chords = flag_notes_chords[2]
    X = []
    Y = []
    X, Y = create_X_Y(chords)
    X = norm_array(X, 4)
    Y = norm_array(Y, 4)
    notes_to_np(notes)
    unique_chords = create_unique_chords(X, unique_chords)
    unique_chords = create_unique_chords(Y, unique_chords)
    unique_chords = unique_chords.reshape((-1, 4))
    unique_chords = np.unique(unique_chords, axis=0)
    unique_chords = list(unique_chords)
    coded_X = []
    coded_Y = []
    coded_X = create_coded(X)
    coded_Y = create_coded(Y)
    encode(X, coded_X)
    encode(Y, coded_Y)
    X = np.array(coded_X)
    y = np.array(coded_Y)
    X_notes = notes
    X_notes = X_notes[:len(X)]
    X = X[:len(X_notes)]
    y = y[:len(X)]
    allX.append([X, X_notes])
    allY.append(y)

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(allX, allY)

In [ ]:
all_len = 0
acc = 0
for x, y in zip(X_train, y_train):
    merged_model.fit(x, y, epochs=1, batch_size=1, verbose=1)
    acc += merged_model.evaluate(x, y, batch_size=1)[1] * len(y)
    all_len += len(y)
    print("current_accuracy: ", acc / all_len)
    merged_model.reset_states()

In [ ]:
y_pred = []
for x, y in zip(X_test, y_test):
    y_pred.append(merged_model.predict(x, batch_size=1, verbose=1).argmax(axis=1))

In [73]:
y_real = y_test

In [74]:
from sklearn.metrics import accuracy_score

test_acc = 0
test_len = 0

for y_r, y_p in zip(y_real, y_pred):
    test_acc += accuracy_score(y_r, y_p) * len(y_r)
    test_len += len(y_r)
    
print("test_accuracy:", test_acc / test_len)

test_accuracy: 0.4337806301050175


In [76]:
df = pd.DataFrame(unique_chords)
df.to_csv(DATASET_HOME+"simple/vocab.csv")

In [77]:
x = pd.read_csv(DATASET_HOME+"simple/vocab.csv")
x = x.values.T[1:].T

In [78]:
x

array([[-1., -1., -1., -1.],
       [ 0., -1., -1., -1.],
       [ 0.,  3.,  8., -1.],
       [ 0.,  4.,  7., -1.],
       [ 0.,  5.,  9., -1.],
       [ 1.,  4.,  9., -1.],
       [ 1.,  5.,  8., -1.],
       [ 1.,  6., 10., -1.],
       [ 2.,  5., 10., -1.],
       [ 2.,  6.,  9., -1.],
       [ 2.,  7., 11., -1.],
       [ 3.,  6., 11., -1.],
       [ 3.,  7., 10., -1.],
       [ 4.,  8., 11., -1.]])

In [ ]:
merged_model.save(DATASET_HOME+"simple/NN_model.h5")